In [1]:
# 1. Download the actual RAW python file, not the .git link
!wget https://raw.githubusercontent.com/aburdenko/bigquery_tool/main/bigquery_tool.py -O bigquery_tool.py

# 2. Add current directory to path
import sys
import os
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

# 3. Load the extension
%reload_ext bigquery_tool

--2026-02-17 23:41:34--  https://raw.githubusercontent.com/aburdenko/bigquery_tool/main/bigquery_tool.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5408 (5.3K) [text/plain]
Saving to: ‘bigquery_tool.py’

bigquery_tool.py    100%[===================>]   5.28K  --.-KB/s    in 0s      

2026-02-17 23:41:34 (52.4 MB/s) - ‘bigquery_tool.py’ saved [5408/5408]



/usr/local/lib/python3.10/dist-packages/google/api_core/_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.12) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/google/api_core/_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.12) which Google will stop supporting in new releases of google.cloud.bigquery_storage_v1 once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.cloud.bigquery_storage_v1 past that date.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/google/api_core/_python_version_support.py:275: FutureWa

In [6]:
import bigframes.pandas as bpd
import ipywidgets as widgets
import plotly.express as px
import plotly.io as pio
import pandas as pd
from IPython.core.magic import register_cell_magic
from IPython.display import display, clear_output
from google.colab import output

output.enable_custom_widget_manager()
pio.renderers.default = "colab"

@register_cell_magic
def bigquery_tool(line, cell):
    table_id = line.strip()
    user_prompt = cell.strip().lower()

    # --- 1. UI Elements ---
    refresh_btn = widgets.Button(description="🔄 Sync Data", button_style='primary')
    filter_by = widgets.Dropdown(description='Filter By:')
    filter_val = widgets.Text(description='Value:', placeholder='e.g. active')
    select_by = widgets.SelectMultiple(description='Select By:', layout={'height': '100px', 'width': '35%'})
    agg_func = widgets.Dropdown(options=['none', 'count', 'mean', 'sum', 'min', 'max'], value='none', description='Aggregate:')
    output_type = widgets.Dropdown(
        options=['Tabular Data', 'Bar Chart', 'Scatter Plot', 'Histogram', 'Box Plot'],
        value='Tabular Data', description='Output Type:'
    )

    control_ui = widgets.VBox([
        widgets.HBox([refresh_btn, filter_by, filter_val]),
        widgets.HBox([select_by, agg_func, output_type]),
    ], layout=widgets.Layout(padding='15px', border='1px solid #555', border_radius='10px'))

    out_area, sql_area = widgets.Output(), widgets.Output()
    current_df = [None]

    def update_viz(change=None):
        with out_area:
            clear_output(wait=True) # Clears the area to prevent double-printing
            df = current_df[0]
            if df is None or not select_by.value: return

            try:
                view_df = df.copy()
                targets = list(select_by.value)
                if filter_val.value and filter_by.value:
                    view_df = view_df[view_df[filter_by.value].astype(str).str.contains(filter_val.value, case=False)]

                func = agg_func.value

                # --- 2. Aggregation Pipeline ---
                if func == 'count':
                    plot_df = view_df.groupby(targets).size().reset_index(name='count_records')
                    y_axis = 'count_records'
                    sql_query = f"SELECT {', '.join(targets)}, COUNT(*) AS count_records FROM `{table_id}` GROUP BY {', '.join(targets)}"

                elif func != 'none':
                    main_measure = targets[-1]
                    view_df[main_measure] = pd.to_numeric(view_df[main_measure], errors='coerce')
                    y_axis = f"{func}_{main_measure}"
                    series_res = view_df.groupby(targets)[main_measure].agg(func)
                    plot_df = series_res.reset_index(name=y_axis)
                    sql_query = f"SELECT {', '.join(targets)}, {func.upper()}({main_measure}) AS {y_axis} FROM `{table_id}` GROUP BY {', '.join(targets)}"

                else:
                    plot_df = view_df[targets]
                    y_axis = targets[-1] if len(targets) > 1 else targets[0]
                    sql_query = f"SELECT {', '.join(targets)} FROM `{table_id}`"

                if len(targets) > 1 and func != 'none':
                    plot_df['Group Combination'] = plot_df[targets].astype(str).agg(' | '.join, axis=1)
                    x_axis = 'Group Combination'
                else:
                    x_axis = targets[0]

                with sql_area:
                    clear_output(wait=True)
                    display(widgets.HTML(f"<pre style='background:#1e1e1e; color:#85c1e9; padding:10px;'>{sql_query}</pre>"))

                # --- 3. Unified Rendering (Fixes Double-Print) ---
                if output_type.value == 'Tabular Data':
                    display(plot_df)
                else:
                    if output_type.value == 'Scatter Plot':
                        fig = px.scatter(plot_df, x=targets[0], y=y_axis)
                    elif output_type.value == 'Box Plot':
                        fig = px.box(plot_df, y=y_axis)
                    elif output_type.value == 'Histogram':
                        fig = px.histogram(plot_df, x=x_axis)
                    else:
                        fig = px.bar(plot_df, x=x_axis, y=y_axis)

                    fig.update_layout(height=400, template="plotly_white", xaxis_title=x_axis)
                    fig.show()

            except Exception as e:
                print(f"Viz Error: {e}")

    def load_data(b=None):
        with out_area:
            # Load fresh data and reset UI state
            try:
                current_df[0] = bpd.read_gbq(table_id).head(500).to_pandas()
                all_cols = current_df[0].columns.tolist()
                filter_by.options = [''] + all_cols
                select_by.options = all_cols

                norm_prompt = user_prompt.replace(" ", "")
                matched_cols = [c for c in all_cols if c.lower() in norm_prompt or c.lower() in user_prompt]
                if matched_cols: select_by.value = tuple(matched_cols)

                if any(k in user_prompt for k in ["dist", "count", "max", "mean"]):
                    agg_func.value = 'count' if "count" in user_prompt or "dist" in user_prompt else 'mean'
                    output_type.value = 'Bar Chart'
                else:
                    agg_func.value, output_type.value = 'none', 'Tabular Data'

                update_viz() # This handles the final display alone
            except Exception as e: print(f"Query Error: {e}")

    refresh_btn.on_click(load_data)
    for w in [filter_by, filter_val, select_by, agg_func, output_type]: w.observe(update_viz, names='value')
    display(control_ui, sql_area, out_area)
    load_data()

In [7]:
%%bigquery_tool kallogjeri-project-345114.bio_demos.gwas_duroc_phenotypes
Show me the count of LiveWeight, sex

Output()

Output()

In [46]:
%%bigquery df
SELECT sex FROM `kallogjeri-project-345114.bio_demos.gwas_duroc_phenotypes`


Query is running:   0%|          |

Downloading:   0%|          |

In [52]:
print(df)


      sex
0    male
1    male
2    male
3    male
4    male
..    ...
347  male
348  male
349  male
350  male
351  male

[352 rows x 1 columns]
